# Imports

In [1]:
import pandas as pd
from utils_association_rules import dataframe_to_transaction_items, creates_associations_rules
import warnings


warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.2f}'.format
pd.set_option("display.max_columns", None)

In [2]:
%cd ..
clusters_data = pd.read_csv('datasets/Final_classified_data.csv', index_col = 0)
basket_data = pd.read_csv('datasets/customer_basket.csv', index_col = 0)
%cd Association-Rules

c:\Users\luisb\OneDrive\Documentos\uni\Year 2\Semester 2\Machine learning 2\ML2_project
c:\Users\luisb\OneDrive\Documentos\uni\Year 2\Semester 2\Machine learning 2\ML2_project\Association-Rules


# Introduction

This notebook will detail the process of obtaining the association rules and campaigns for each cluster. 

Firstly, we'll just take a look at the datasets that will be used:
+ **clusters_data** includes customers, their shopping behaviour and the cluster that they were assigned.
+ **basket_data** includes customers and the items they bought.

In [3]:
clusters_data

,kids_home,teens_home,number_complaints,distinct_stores_visited,pct_spend_groceries,pct_spend_electronics,typical_hour,pct_spend_vegetables,pct_spend_nonalcohol_drinks,pct_spend_alcohol_drinks,pct_spend_meat,pct_spend_fish,pct_spend_hygiene,pct_spend_videogames,pct_spend_petfood,lifetime_total_distinct_products,percentage_of_products_bought_promotion,Age,male,loyalty_card,total_spend,customer_tenure,customer_name,Bsc.,Hsc.,Msc.,Phd.,Final_Cluster
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
48086,0,0,1,1,23.32,0.08,7.00,0.39,0.45,0.18,0.14,75.32,0.13,0.00,0.00,14.00,24.77,58.00,1.00,0.00,64304.00,13.00,Charles`s Fishy,0,1,0,0,Fishermen
31838,0,0,1,1,22.86,0.17,9.00,0.53,0.74,0.06,0.05,75.42,0.17,0.00,0.00,24.00,0.00,61.00,0.00,1.00,60528.00,13.00,Ursula`s Fishy,0,1,0,0,Fishermen
41519,0,0,0,1,27.14,0.31,9.00,0.21,0.52,0.15,0.16,71.23,0.27,0.00,0.00,33.00,39.93,68.00,0.00,0.00,54085.00,9.00,Lindsay`s Fishy,0,1,0,0,Fishermen
41076,0,0,0,1,29.92,0.71,12.00,0.82,0.50,0.14,0.35,67.40,0.16,0.00,0.00,34.00,28.27,82.00,0.00,0.00,50499.00,14.00,Carol`s Fishy,0,1,0,0,Fishermen
43120,0,0,1,1,29.64,0.35,9.00,0.55,0.61,0.19,0.13,68.49,0.03,0.00,0.00,20.00,39.62,44.00,0.00,0.00,57166.00,13.00,Rosetta`s Fishy,0,1,0,0,Fishermen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36217,2,1,1,1,34.88,19.23,18.00,4.00,2.90,2.07,10.66,3.80,6.20,8.05,8.21,744.00,19.42,67.00,0.00,0.00,12013.00,16.00,Patricia Garver,0,1,0,0,Parents
32758,3,2,2,1,33.47,15.77,19.00,3.70,5.28,1.14,4.32,6.54,8.14,15.08,6.56,133.00,18.63,46.00,0.00,0.00,11094.00,17.00,Carman Stubblefield,0,1,0,0,Parents
22316,0,2,2,1,36.19,17.77,9.00,2.78,3.44,3.78,10.76,7.20,4.57,4.60,8.92,353.00,28.01,82.00,0.00,1.00,13314.00,13.00,Betty Nance,0,1,0,0,Parents


In [4]:
basket_data

,list_of_goods,customer_id
invoice_id,,
7448187,"['olive oil', 'yogurt cake', 'grated cheese', ...",43134
3167314,"['babies food', 'shower gel', 'rice', 'black b...",43134
10199712,"['babies food', 'barbecue sauce', 'olive oil',...",43134
12179179,"['babies food', 'milk', 'rice', 'yogurt cake',...",43134
8292388,"['napkins', 'cookies', 'razor', 'cooking oil',...",43134
...,...,...
10920856,"['rice', 'pet food', 'cauliflower', 'cake', 'c...",31335
10239983,"['cotton buds', 'cream', 'gums', 'cooking oil'...",35452
10238942,"['cottage cheese', 'champagne', 'spaghetti', '...",3608


In [5]:
basket_data[basket_data[['customer_id']].duplicated()]

,list_of_goods,customer_id
invoice_id,,
3167314,"['babies food', 'shower gel', 'rice', 'black b...",43134
10199712,"['babies food', 'barbecue sauce', 'olive oil',...",43134
12179179,"['babies food', 'milk', 'rice', 'yogurt cake',...",43134
8292388,"['napkins', 'cookies', 'razor', 'cooking oil',...",43134
11607908,"['green beans', 'toilet paper', 'yogurt cake',...",43134
...,...,...
4445879,"['babies food', 'napkins', 'tomato sauce', 'fi...",38914
5009540,"['milk', 'google tablet', 'red wine', 'bramble...",20971
2426016,"['babies food', 'rice', 'napkins', 'gums', 'fr...",21637


We have multiple transactions per customer, as expected.

In [6]:
basket_data = basket_data.reset_index()
basket_data[basket_data[['invoice_id']].duplicated(keep=False)]


,invoice_id,list_of_goods,customer_id
15,7469041,"['milk', 'strong cheese', 'cotton buds', 'butt...",8251
323,4695137,"['green beans', 'tomatoes', 'olive oil', 'iPad...",17499
327,2674958,"['green beans', 'tomatoes', 'burgers', 'carrot...",9665
330,2674958,"['green beans', 'tomatoes', 'burgers', 'carrot...",26604
569,2877777,"['salmon', 'shallot', 'airpods', 'escalope', '...",31964
...,...,...,...
87780,4488958,"['shower gel', 'napkins', 'chocolate bread', '...",25630
88139,9079941,"['green beans', 'shower gel', 'melons', 'aspar...",21116
88280,7614681,"['turkey', 'ground beef', 'cooking oil', 'cake...",43362
88821,2520657,"['white wine', 'french wine', 'seabass', 'oil']",26472


There are some transactions with the same invoice_id and different customer_id, so we need to treat those.

In [7]:
basket_data[basket_data[['invoice_id', 'customer_id']].duplicated()]

,invoice_id,list_of_goods,customer_id


Since there are only 602 transactions with this issue we will remove those, because a transaction cannot be the same for 2 customers and we dont know to whom the transaction belongs in reality.

In [8]:
basket_data.drop_duplicates(subset=['invoice_id'], keep=False, inplace=True, ignore_index=True)

In [9]:
basket_data[basket_data[['invoice_id']].duplicated()]


,invoice_id,list_of_goods,customer_id


Now that there are no issues with duplicates, we can set the invoice_id as the index.

In [10]:
basket_data.set_index('invoice_id', inplace=True)
basket_data

,list_of_goods,customer_id
invoice_id,,
7448187,"['olive oil', 'yogurt cake', 'grated cheese', ...",43134
3167314,"['babies food', 'shower gel', 'rice', 'black b...",43134
10199712,"['babies food', 'barbecue sauce', 'olive oil',...",43134
12179179,"['babies food', 'milk', 'rice', 'yogurt cake',...",43134
8292388,"['napkins', 'cookies', 'razor', 'cooking oil',...",43134
...,...,...
10920856,"['rice', 'pet food', 'cauliflower', 'cake', 'c...",31335
10239983,"['cotton buds', 'cream', 'gums', 'cooking oil'...",35452
10238942,"['cottage cheese', 'champagne', 'spaghetti', '...",3608


# Dividing the Baskets

We divide the basket per cluster to create separate rules per type of client.

In [11]:
# Each basket is the transactions made by an especific cluster group.

basket_Fishermen = basket_data[basket_data['customer_id'].isin(clusters_data[clusters_data['Final_Cluster'] == 'Fishermen'].index)]

basket_Gamers = basket_data[basket_data['customer_id'].isin(clusters_data[clusters_data['Final_Cluster'] == 'Gamers'].index)]

basket_Pet_lovers = basket_data[basket_data['customer_id'].isin(clusters_data[clusters_data['Final_Cluster'] == 'Pet lovers'].index)]

basket_Young_party_people = basket_data[basket_data['customer_id'].isin(clusters_data[clusters_data['Final_Cluster'] == 'Young party people'].index)]

basket_Vegetarians = basket_data[basket_data['customer_id'].isin(clusters_data[clusters_data['Final_Cluster'] == 'Vegetarians'].index)]

basket_Young_electronics = basket_data[basket_data['customer_id'].isin(clusters_data[clusters_data['Final_Cluster'] == 'Young, lots of electronics'].index)]

basket_Loyal_Customers = basket_data[basket_data['customer_id'].isin(clusters_data[clusters_data['Final_Cluster'] == 'Loyal Customers'].index)]

basket_Promotions = basket_data[basket_data['customer_id'].isin(clusters_data[clusters_data['Final_Cluster'] == 'Promotions'].index)]

basket_Parents = basket_data[basket_data['customer_id'].isin(clusters_data[clusters_data['Final_Cluster'] == 'Parents'].index)]

It's important to clarify that the rules that are going to be created consider the behavior of the group (learned in the cluster analysis) and also the past transactions made by individuals in those groups. For logical reasons usual items like oil, water, napkins or milk won’t be consider as valuable since they could be consider as essential products, and for that reason appear in a large majority of transactions.

**The promotions don’t need to be implemented at the same time, since the ones that are going to be suggested involve  a certain product multiple times.**

## Basket Fishermen

In [12]:
basket_Fishermen
# info of 905 transactions

,list_of_goods,customer_id
invoice_id,,
11985201,"['shrimp', 'napkins', 'yogurt cake', 'chili', ...",4104
10096910,"['strong cheese', 'yogurt cake', 'gums', 'salm...",4104
11572471,"['shrimp', 'rice', 'cotton buds', 'napkins', '...",4104
11917820,"['shrimp', 'whole weat flour', 'cookies', 'sal...",4104
11556833,"['shrimp', 'black beer', 'whole weat flour', '...",4104
...,...,...
3732026,"['shrimp', 'blueberries', 'whole weat flour', ...",31173
10082353,"['shrimp', 'melons', 'gums', 'salmon', 'soup',...",23002
5104467,"['gluten free bar', 'trout', 'strong cheese', ...",27266


In [13]:
transactions_Fishermen = dataframe_to_transaction_items(basket_Fishermen)
transactions_Fishermen

,airpods,almonds,antioxydantjuice,asparagus,avocado,babiesfood,bacon,barbecuesauce,beatsheadphones,beer,blackbeer,blacktea,blueberries,bluetoothheadphones,bodyspray,bramble,brownies,burgers,burgersauce,butter,cake,candybars,canned_tuna,carrots,catfish,cauliflower,cereals,champagne,chicken,chili,chocolate,chocolatebread,chutney,cider,cologne,cookies,cookingoil,corn,cottagecheese,cottonbuds,cream,deodorant,dessertwine,eggplant,eggs,energybar,energydrink,escalope,extradarkchocolate,finalfantasyXIX,finalfantasyXX,finalfantasyXXII,flaxseed,frenchfries,frenchwine,freshbread,freshtuna,fromageblanc,frozensmoothie,frozenvegetables,glutenfreebar,googletablet,gratedcheese,greenbeans,greengrapes,greentea,groundbeef,gums,half-life2,half-life:alyx,ham,handproteinbar,herb&pepper,honey,hotdogs,iMac,iPad,iphone8,ketchup,laptop,lightcream,lightmayo,lowfatyogurt,mashedpotato,mayonnaise,meatballs,megamanzero,megamanzero2,megamanzero3,megamanzero4,melons,metroidfusion,metroidprime,milk,minecraft,mineralwater,mint,mintgreentea,muffins,mushroomcreamsauce,napkins,nonfatmilk,oatmeal,oil,oliveoil,pancakes,parmesancheese,pasta,pepper,petfood,phonecarcharger,phonecharger,pickles,pokemonscarlet,pokemonshield,pokemonsword,pokemonviolet,portal,portal2,proteinbar,ratchet&clank,ratchet&clank2,ratchet&clank3,razor,redwine,rice,salad,salmon,salt,samsunggalaxy10,sandwich,seabass,shallot,shampoo,showergel,shrimp,soda,soup,spaghetti,sparklingwater,spinach,strawberries,strongcheese,tea,toiletpaper,tomatoes,tomatojuice,tomatosauce,toothbrush,toothpaste,trout,turkey,vacuumcleaner,vegetablesmix,waterspray,whitewine,wholeweatflour,wholewheatpasta,wholewheatrice,yams,yogurtcake,zucchini
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,Fals

In [14]:
creates_associations_rules(transaction_items=transactions_Fishermen, min_support=0.12, min_confidence=0.5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
21,(catfish),"(shrimp, freshtuna)",0.24,0.45,0.12,0.51,1.15,0.02,1.14,0.17
5,(frenchfries),(shrimp),0.15,0.78,0.13,0.88,1.12,0.01,1.78,0.13
20,"(catfish, shrimp)",(freshtuna),0.20,0.55,0.12,0.62,1.12,0.01,1.17,0.13
24,"(shrimp, seabass)",(freshtuna),0.23,0.55,0.14,0.61,1.11,0.01,1.16,0.13
19,"(catfish, freshtuna)",(shrimp),0.14,0.78,0.12,0.86,1.10,0.01,1.59,0.11
17,"(freshtuna, canned_tuna)",(shrimp),0.18,0.78,0.15,0.86,1.10,0.01,1.54,0.11
22,"(shrimp, salmon)",(freshtuna),0.34,0.55,0.21,0.60,1.09,0.02,1.13,0.13
6,(salmon),(freshtuna),0.43,0.55,0.26,0.60,1.08,0.02,1.11,0.13
25,"(freshtuna, seabass)",(shrimp),0.17,0.78,0.14,0.84,1.07,0.01,1.35,0.08
2,(catfish),(freshtuna),0.24,0.55,0.14,0.59,1.07,0.01,1.10,0.09


In [15]:
creates_associations_rules(transaction_items=transactions_Fishermen, min_support=0.05, min_confidence=0.5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
156,"(freshtuna, oil)","(shrimp, salmon)",0.11,0.34,0.06,0.50,1.47,0.02,1.33,0.36
153,"(shrimp, freshtuna, oil)",(salmon),0.10,0.43,0.06,0.60,1.40,0.02,1.42,0.31
111,"(freshtuna, oil)",(salmon),0.11,0.43,0.07,0.59,1.38,0.02,1.40,0.31
119,(petfood),"(shrimp, freshtuna)",0.10,0.45,0.06,0.59,1.33,0.01,1.36,0.28
64,(babiesfood),"(shrimp, freshtuna)",0.11,0.45,0.07,0.58,1.31,0.02,1.33,0.27
140,"(shrimp, soup)",(salmon),0.10,0.43,0.05,0.55,1.29,0.01,1.27,0.25
93,(frenchfries),"(shrimp, freshtuna)",0.15,0.45,0.09,0.57,1.27,0.02,1.28,0.25
157,"(salmon, oil)","(shrimp, freshtuna)",0.10,0.45,0.06,0.57,1.27,0.01,1.28,0.24
137,"(shrimp, oil)",(salmon),0.16,0.43,0.09,0.54,1.26,0.02,1.24,0.24
22,(milk),(freshtuna),0.12,0.55,0.08,0.70,1.26,0.02,1.46,0.23


We had 905 transactions made by the fishermen group, we considered 2 possible sets of rules, one with products that appear at least 12% of the time and 5% of the time, appearing at least 50% of the time together.
After looking at the results we see that:
+ Shrimp is bought with almost everything, meaning that they go to the place which has the best price on shrimp.
+ Tuna and Salmon appear together often in different transactions.
+ They mostly bought fish products.

We suggest the following promotions and strategies to encourage purchases in this group:
+ To incentivize higher purchases: 10% discount on fish products in purchases before 1pm.
+ To incentivize the joint purchase of fresh tuna and salmon: 10% discount of in fresh tuna and salmon if they are bought together.
+ To ensure that they keep buying in our store: 5% of discount if you bought shrimp.

Promotions are delivered using the loyalty card.


## Basket Gamers

In [16]:
basket_Gamers
# info of 12873 transactions

,list_of_goods,customer_id
invoice_id,,
8239601,"['turkey', 'phone car charger', 'bluetooth hea...",29810
3232148,"['cottage cheese', 'spaghetti', 'carrots', 'sh...",29810
7345022,"['shower gel', 'shrimp', 'gluten free bar', 'g...",29810
3279772,"['turkey', 'beats headphones', 'bluetooth head...",29810
3009879,"['shrimp', 'parmesan cheese', 'samsung galaxy ...",29810
...,...,...
3415021,"['shower gel', 'shrimp', 'parmesan cheese', 's...",29532
1814718,"['shower gel', 'body spray']",19735
6329789,"['chutney', 'phone car charger', 'laptop', 'ch...",39702


In [17]:
transactions_Gamers = dataframe_to_transaction_items(basket_Gamers)
transactions_Gamers

,airpods,almonds,antioxydantjuice,asparagus,avocado,babiesfood,bacon,barbecuesauce,beatsheadphones,beer,blackbeer,blacktea,blueberries,bluetoothheadphones,bodyspray,bramble,brownies,burgers,burgersauce,butter,cake,candybars,canned_tuna,carrots,catfish,cauliflower,cereals,champagne,chicken,chili,chocolate,chocolatebread,chutney,cider,cologne,cookies,cookingoil,corn,cottagecheese,cottonbuds,cream,deodorant,dessertwine,eggplant,eggs,energybar,energydrink,escalope,extradarkchocolate,finalfantasyXIX,finalfantasyXX,finalfantasyXXII,flaxseed,frenchfries,frenchwine,freshbread,freshtuna,fromageblanc,frozensmoothie,frozenvegetables,glutenfreebar,googletablet,gratedcheese,greenbeans,greengrapes,greentea,groundbeef,gums,half-life2,half-life:alyx,ham,handproteinbar,herb&pepper,honey,hotdogs,iMac,iPad,iphone8,ketchup,laptop,lightcream,lightmayo,lowfatyogurt,mashedpotato,mayonnaise,meatballs,megamanzero,megamanzero2,megamanzero3,megamanzero4,melons,metroidfusion,metroidprime,milk,minecraft,mineralwater,mint,mintgreentea,muffins,mushroomcreamsauce,napkins,nonfatmilk,oatmeal,oil,oliveoil,pancakes,parmesancheese,pasta,pepper,petfood,phonecarcharger,phonecharger,pickles,pokemonscarlet,pokemonshield,pokemonsword,pokemonviolet,portal,portal2,proteinbar,ratchet&clank,ratchet&clank2,ratchet&clank3,razor,redwine,rice,salad,salmon,salt,samsunggalaxy10,sandwich,seabass,shallot,shampoo,showergel,shrimp,soda,soup,spaghetti,sparklingwater,spinach,strawberries,strongcheese,tea,toiletpaper,tomatoes,tomatojuice,tomatosauce,toothbrush,toothpaste,trout,turkey,vacuumcleaner,vegetablesmix,waterspray,whitewine,wholeweatflour,wholewheatpasta,wholewheatrice,yams,yogurtcake,zucchini
0,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False,Fal

In [18]:
creates_associations_rules(transaction_items=transactions_Gamers, min_support=0.15, min_confidence=0.5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
12,"(champagne, airpods)",(samsunggalaxy10),0.27,0.51,0.16,0.57,1.13,0.02,1.15,0.15
14,"(champagne, bluetoothheadphones)",(samsunggalaxy10),0.31,0.51,0.17,0.56,1.11,0.02,1.12,0.14
15,"(bluetoothheadphones, samsunggalaxy10)",(champagne),0.21,0.74,0.17,0.82,1.10,0.02,1.42,0.12
13,"(airpods, samsunggalaxy10)",(champagne),0.19,0.74,0.16,0.81,1.10,0.01,1.40,0.11
11,(spaghetti),(samsunggalaxy10),0.30,0.51,0.17,0.55,1.09,0.01,1.10,0.11
1,(airpods),(samsunggalaxy10),0.35,0.51,0.19,0.55,1.08,0.01,1.09,0.11
6,(iphone8),(champagne),0.22,0.74,0.18,0.79,1.07,0.01,1.23,0.08
7,(laptop),(champagne),0.26,0.74,0.20,0.79,1.07,0.01,1.23,0.08
3,(bluetoothheadphones),(samsunggalaxy10),0.39,0.51,0.21,0.54,1.06,0.01,1.07,0.10
5,(fromageblanc),(champagne),0.22,0.74,0.17,0.79,1.06,0.01,1.21,0.07


In [19]:
creates_associations_rules(transaction_items=transactions_Gamers, min_support=0.08, min_confidence=0.5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
57,"(champagne, laptop)",(samsunggalaxy10),0.20,0.51,0.12,0.58,1.13,0.01,1.16,0.15
59,"(champagne, spaghetti)",(samsunggalaxy10),0.24,0.51,0.14,0.58,1.13,0.02,1.16,0.15
37,"(airpods, bluetoothheadphones)",(samsunggalaxy10),0.15,0.51,0.09,0.57,1.13,0.01,1.15,0.13
39,"(champagne, airpods)",(samsunggalaxy10),0.27,0.51,0.16,0.57,1.13,0.02,1.15,0.15
56,"(samsunggalaxy10, iphone8)",(champagne),0.12,0.74,0.10,0.83,1.12,0.01,1.51,0.12
43,"(bluetoothheadphones, laptop)",(champagne),0.11,0.74,0.09,0.82,1.11,0.01,1.48,0.12
51,"(champagne, gratedcheese)",(samsunggalaxy10),0.15,0.51,0.08,0.56,1.11,0.01,1.13,0.12
38,"(airpods, laptop)",(champagne),0.10,0.74,0.08,0.82,1.11,0.01,1.46,0.11
58,"(samsunggalaxy10, laptop)",(champagne),0.14,0.74,0.12,0.82,1.11,0.01,1.45,0.12
50,"(fromageblanc, samsunggalaxy10)",(champagne),0.11,0.74,0.09,0.82,1.11,0.01,1.45,0.11


We had 12873 transactions made by the gamers group, we considered 2 possible sets of rules, one with products that appear at least 15% of the time and 8% of the time, appearing at least 50% of the time together.
After looking at the results we see that:
+ These people bought laptops, Samsung galaxy or iPhone.
+ Champagne is often bought with electronics.
+ Headphones are often bought with a phone.
+ They mostly bought electronic products.

We suggest the following promotions and strategies to encourage purchases in this group:
+ Create catalogs (digital or physical) about new games, phones or laptops and send them to these customers.
+ To incentivize the purchase of champagne on the purchase of a phone or laptop: 50% discount on champagne with a purchase of a laptop, Samsung galaxy or iPhone.
+ To incentivize them to by headphones when buying a phone: 30% discount on Bluetooth headphones if you bought a Samsung galaxy (do the same with airpods and an iPhone).

Promotions are delivered using the loyalty card and in ad campaigns.


## Basket Pet lovers

In [20]:
basket_Pet_lovers
# info of 0  transactions

,list_of_goods,customer_id
invoice_id,,


We don’t have any registered transactions made by this type of customers, but we know that they don’t have children, spend the majority on pets' products, so we recommend the following promotions and strategies  to encourage purchases in this group:
+ Create catalogs (digital or physical) about toys, homes and food for pets (being generic about the type of pet) and send them to these customers.
+ To keep them buying in our store: 10% discount on pet products on purchases higher than 50€.
+ To incentivize the purchase of new toys: Buy 2 toys for your pet and get 1 free.

Promotions are delivered in ad campaigns and tv commercials.



## Basket Young party people

In [21]:
basket_Young_party_people
# info of 2255 transactions

,list_of_goods,customer_id
invoice_id,,
4191670,"['beer', 'fresh bread', 'zucchini', 'dessert w...",42201
4943054,"['cookies', 'gums', 'fresh bread', 'salad', 'f...",42201
10067264,"['low fat yogurt', 'bacon', 'white wine', 'bur...",42201
2787047,"['babies food', 'rice', 'whole weat flour', 'c...",42201
9980938,"['oatmeal', 'white wine', 'black tea', 'ham', ...",30841
...,...,...
1983362,"['beer', 'oatmeal', 'white wine', 'soda', 'coo...",34406
5646784,"['whole wheat rice', 'cider', 'chocolate bread...",25990
7225640,"['cake', 'bramble', 'white wine', 'cooking oil']",19524


In [22]:
transactions_Young_party_people = dataframe_to_transaction_items(basket_Young_party_people)
transactions_Young_party_people

,airpods,almonds,antioxydantjuice,asparagus,avocado,babiesfood,bacon,barbecuesauce,beatsheadphones,beer,blackbeer,blacktea,blueberries,bluetoothheadphones,bodyspray,bramble,brownies,burgers,burgersauce,butter,cake,candybars,canned_tuna,carrots,catfish,cauliflower,cereals,champagne,chicken,chili,chocolate,chocolatebread,chutney,cider,cologne,cookies,cookingoil,corn,cottagecheese,cottonbuds,cream,deodorant,dessertwine,eggplant,eggs,energybar,energydrink,escalope,extradarkchocolate,finalfantasyXIX,finalfantasyXX,finalfantasyXXII,flaxseed,frenchfries,frenchwine,freshbread,freshtuna,fromageblanc,frozensmoothie,frozenvegetables,glutenfreebar,googletablet,gratedcheese,greenbeans,greengrapes,greentea,groundbeef,gums,half-life2,half-life:alyx,ham,handproteinbar,herb&pepper,honey,hotdogs,iMac,iPad,iphone8,ketchup,laptop,lightcream,lightmayo,lowfatyogurt,mashedpotato,mayonnaise,meatballs,megamanzero,megamanzero2,megamanzero3,megamanzero4,melons,metroidfusion,metroidprime,milk,minecraft,mineralwater,mint,mintgreentea,muffins,mushroomcreamsauce,napkins,nonfatmilk,oatmeal,oil,oliveoil,pancakes,parmesancheese,pasta,pepper,petfood,phonecarcharger,phonecharger,pickles,pokemonscarlet,pokemonshield,pokemonsword,pokemonviolet,portal,portal2,proteinbar,ratchet&clank,ratchet&clank2,ratchet&clank3,razor,redwine,rice,salad,salmon,salt,samsunggalaxy10,sandwich,seabass,shallot,shampoo,showergel,shrimp,soda,soup,spaghetti,sparklingwater,spinach,strawberries,strongcheese,tea,toiletpaper,tomatoes,tomatojuice,tomatosauce,toothbrush,toothpaste,trout,turkey,vacuumcleaner,vegetablesmix,waterspray,whitewine,wholeweatflour,wholewheatpasta,wholewheatrice,yams,yogurtcake,zucchini
0,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,True,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [23]:
creates_associations_rules(transaction_items=transactions_Young_party_people, min_support=0.1, min_confidence=0.5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
16,"(whitewine, dessertwine)",(cider),0.26,0.47,0.15,0.56,1.19,0.02,1.20,0.22
13,"(beer, whitewine)",(cider),0.23,0.47,0.13,0.55,1.16,0.02,1.17,0.18
7,(dessertwine),(cider),0.34,0.47,0.18,0.54,1.13,0.02,1.14,0.18
0,(beer),(cider),0.30,0.47,0.16,0.54,1.13,0.02,1.14,0.17
15,"(cider, dessertwine)",(whitewine),0.18,0.72,0.15,0.81,1.13,0.02,1.46,0.14
14,"(beer, cider)",(whitewine),0.16,0.72,0.13,0.81,1.12,0.01,1.46,0.13
3,(bramble),(cider),0.19,0.47,0.10,0.53,1.11,0.01,1.11,0.13
1,(beer),(whitewine),0.30,0.72,0.23,0.79,1.10,0.02,1.33,0.13
12,(redwine),(whitewine),0.16,0.72,0.12,0.77,1.07,0.01,1.23,0.08
10,(dessertwine),(whitewine),0.34,0.72,0.26,0.77,1.07,0.02,1.23,0.10


In [24]:
creates_associations_rules(transaction_items=transactions_Young_party_people, min_support=0.05, min_confidence=0.5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
51,"(beer, dessertwine)",(cider),0.11,0.47,0.07,0.59,1.24,0.01,1.27,0.22
73,"(beer, whitewine, dessertwine)",(cider),0.09,0.47,0.05,0.58,1.22,0.01,1.24,0.20
68,"(whitewine, dessertwine)",(cider),0.26,0.47,0.15,0.56,1.19,0.02,1.20,0.22
22,(yogurtcake),(cider),0.10,0.47,0.05,0.56,1.17,0.01,1.18,0.16
61,"(champagne, dessertwine)",(cider),0.09,0.47,0.05,0.55,1.17,0.01,1.18,0.16
71,"(whitewine, redwine)",(cider),0.12,0.47,0.07,0.55,1.16,0.01,1.17,0.16
52,"(beer, whitewine)",(cider),0.23,0.47,0.13,0.55,1.16,0.02,1.17,0.18
54,"(beer, dessertwine)",(whitewine),0.11,0.72,0.09,0.83,1.16,0.01,1.66,0.15
9,(candybars),(cider),0.12,0.47,0.06,0.55,1.16,0.01,1.16,0.15
66,"(cookingoil, whitewine)",(cider),0.09,0.47,0.05,0.55,1.16,0.01,1.16,0.15


We had 2255 transactions made by the young parties group, we considered 2 possible sets of rules, one with products that appear at least 10% of the time and 5% of the time, appearing at least 50% of the time together.
After looking at the results we see that:
+ These people bought mostly cider, white and dessert wine, and beer.
+ They mostly bought alcohol products.

We suggest the following promotions and strategies to encourage purchases in this group:
+ To incentivize higher purchases: 10% discount on alcohol and non-alcohol products on purchases higher than 50€.
+ To incentivize the purchase of other products: 15% discount on beer on the purchase of cider (apply the same discount to white wine and dessert wine).
+ To incentivize them to buy wine products together: 20% discount on dessert wine and white wine if bought together.

Promotions are delivered in ad campaigns and loyalty card.


## Basket Vegetarians

In [25]:
basket_Vegetarians
# info of 16229 transactions

,list_of_goods,customer_id
invoice_id,,
1888809,"['melons', 'pickles', 'tomatoes', 'ratchet & c...",5422
3757702,"['shower gel', 'green beans', 'pepper', 'tomat...",5422
11377936,"['champagne', 'carrots', 'chocolate', 'candy b...",5422
2802811,"['cologne', 'pepper', 'phone car charger', 'po...",14007
11512739,"['bacon', 'tomatoes', 'tomato sauce', 'caulifl...",14007
...,...,...
10054761,"['turkey', 'beats headphones', 'tomatoes', 'st...",40851
6615614,"['bacon', 'melons', 'ground beef', 'tomatoes',...",16885
8560381,"['green beans', 'minecraft', 'bluetooth headph...",43389


In [26]:
transactions_Vegetarians = dataframe_to_transaction_items(basket_Vegetarians)
transactions_Vegetarians

,airpods,almonds,antioxydantjuice,asparagus,avocado,babiesfood,bacon,barbecuesauce,beatsheadphones,beer,blackbeer,blacktea,blueberries,bluetoothheadphones,bodyspray,bramble,brownies,burgers,burgersauce,butter,cake,candybars,canned_tuna,carrots,catfish,cauliflower,cereals,champagne,chicken,chili,chocolate,chocolatebread,chutney,cider,cologne,cookies,cookingoil,corn,cottagecheese,cottonbuds,cream,deodorant,dessertwine,eggplant,eggs,energybar,energydrink,escalope,extradarkchocolate,finalfantasyXIX,finalfantasyXX,finalfantasyXXII,flaxseed,frenchfries,frenchwine,freshbread,freshtuna,fromageblanc,frozensmoothie,frozenvegetables,glutenfreebar,googletablet,gratedcheese,greenbeans,greengrapes,greentea,groundbeef,gums,half-life2,half-life:alyx,ham,handproteinbar,herb&pepper,honey,hotdogs,iMac,iPad,iphone8,ketchup,laptop,lightcream,lightmayo,lowfatyogurt,mashedpotato,mayonnaise,meatballs,megamanzero,megamanzero2,megamanzero3,megamanzero4,melons,metroidfusion,metroidprime,milk,minecraft,mineralwater,mint,mintgreentea,muffins,mushroomcreamsauce,napkins,nonfatmilk,oatmeal,oil,oliveoil,pancakes,parmesancheese,pasta,pepper,petfood,phonecarcharger,phonecharger,pickles,pokemonscarlet,pokemonshield,pokemonsword,pokemonviolet,portal,portal2,proteinbar,ratchet&clank,ratchet&clank2,ratchet&clank3,razor,redwine,rice,salad,salmon,salt,samsunggalaxy10,sandwich,seabass,shallot,shampoo,showergel,shrimp,soda,soup,spaghetti,sparklingwater,spinach,strawberries,strongcheese,tea,toiletpaper,tomatoes,tomatojuice,tomatosauce,toothbrush,toothpaste,trout,turkey,vacuumcleaner,vegetablesmix,waterspray,whitewine,wholeweatflour,wholewheatpasta,wholewheatrice,yams,yogurtcake,zucchini
0,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [27]:
creates_associations_rules(transaction_items=transactions_Vegetarians, min_support=0.15, min_confidence=0.5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
26,(melons),"(tomatoes, asparagus)",0.25,0.55,0.15,0.61,1.12,0.02,1.17,0.15
29,(mashedpotato),"(tomatoes, carrots)",0.35,0.46,0.18,0.51,1.12,0.02,1.11,0.16
24,"(tomatoes, melons)",(asparagus),0.20,0.67,0.15,0.74,1.11,0.02,1.29,0.12
27,"(tomatoes, mashedpotato)",(carrots),0.29,0.56,0.18,0.62,1.11,0.02,1.16,0.14
23,(mashedpotato),"(tomatoes, asparagus)",0.35,0.55,0.21,0.60,1.11,0.02,1.14,0.15
16,"(asparagus, mashedpotato)",(carrots),0.25,0.56,0.15,0.62,1.10,0.01,1.15,0.12
21,"(tomatoes, mashedpotato)",(asparagus),0.29,0.67,0.21,0.73,1.09,0.02,1.24,0.12
15,"(carrots, mashedpotato)",(asparagus),0.21,0.67,0.15,0.73,1.09,0.01,1.23,0.11
28,"(carrots, mashedpotato)",(tomatoes),0.21,0.78,0.18,0.85,1.09,0.01,1.45,0.10
18,"(tomatoes, asparagus)",(carrots),0.55,0.56,0.33,0.61,1.08,0.03,1.12,0.17


In [28]:
creates_associations_rules(transaction_items=transactions_Vegetarians, min_support=0.05, min_confidence=0.5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
222,"(asparagus, greenbeans)","(tomatoes, carrots)",0.13,0.46,0.07,0.55,1.19,0.01,1.20,0.19
248,"(melons, mashedpotato)","(tomatoes, asparagus)",0.09,0.55,0.06,0.65,1.19,0.01,1.29,0.17
221,"(carrots, greenbeans)","(tomatoes, asparagus)",0.11,0.55,0.07,0.65,1.18,0.01,1.29,0.18
245,"(frozenvegetables, mashedpotato)","(tomatoes, asparagus)",0.09,0.55,0.06,0.65,1.18,0.01,1.28,0.17
237,"(asparagus, shallot)","(tomatoes, carrots)",0.10,0.46,0.05,0.54,1.18,0.01,1.18,0.17
231,"(carrots, melons)","(tomatoes, asparagus)",0.15,0.55,0.10,0.64,1.18,0.01,1.27,0.18
251,"(melons, mashedpotato)","(tomatoes, carrots)",0.09,0.46,0.05,0.54,1.17,0.01,1.17,0.16
236,"(carrots, shallot)","(tomatoes, asparagus)",0.08,0.55,0.05,0.64,1.17,0.01,1.26,0.16
227,"(asparagus, mashedpotato)","(tomatoes, carrots)",0.25,0.46,0.13,0.53,1.16,0.02,1.16,0.19
232,"(asparagus, melons)","(tomatoes, carrots)",0.18,0.46,0.10,0.53,1.16,0.01,1.16,0.17


We had 16229 transactions made by the vegetarian’s group, we considered 2 possible sets of rules, one with products that appear at least 15% of the time and 5% of the time, appearing at least 50% of the time together.
After looking at the results we see that:
+ These people bought asparagus, tomatoes, carrots and mashed potatoes with everything. 
+ Melons are bought normally with the other products.
+ They mostly bought vegetable products.

We suggest the following promotions and strategies to encourage purchases in this group:
+ To encourage higher purchases: 10% discount on vegetables products on purchases higher than 30€.
+ To encourage the purchase of other products: 15% discount on melons and mashed potatoes on purchase of 1 Kg (combined) of tomatoes and asparagus.
+ To ensure they keep buying in our store: 10% discount on asparagus, tomatoes, carrots and mashed potatoes.

Promotions are delivered using the loyalty card and in ad campaigns.


## Basket Young, lots of electronics

In [29]:
basket_Young_electronics
# info of 5725 transactions

,list_of_goods,customer_id
invoice_id,,
9089424,"['low fat yogurt', 'babies food', 'pancakes', ...",18917
9415929,"['gums', 'fresh bread', 'ketchup', 'cooking oi...",18917
1607905,"['milk', 'napkins', 'chili', 'cooking oil', 'c...",18917
9448585,"['cake', 'muffins', 'napkins']",18917
5544273,"['shower gel', 'zucchini', 'napkins', 'yogurt ...",18917
...,...,...
6931681,"['bacon', 'samsung galaxy 10', 'napkins', 'pho...",37387
2771566,"['french fries', 'gluten free bar', 'fresh bre...",30966
11269260,"['babies food', 'tooth brush', 'napkins', 'cot...",31447


In [30]:
transactions_Young_electronics = dataframe_to_transaction_items(basket_Young_electronics)
transactions_Young_electronics

,airpods,almonds,antioxydantjuice,asparagus,avocado,babiesfood,bacon,barbecuesauce,beatsheadphones,beer,blackbeer,blacktea,blueberries,bluetoothheadphones,bodyspray,bramble,brownies,burgers,burgersauce,butter,cake,candybars,canned_tuna,carrots,catfish,cauliflower,cereals,champagne,chicken,chili,chocolate,chocolatebread,chutney,cider,cologne,cookies,cookingoil,corn,cottagecheese,cottonbuds,cream,deodorant,dessertwine,eggplant,eggs,energybar,energydrink,escalope,extradarkchocolate,finalfantasyXIX,finalfantasyXX,finalfantasyXXII,flaxseed,frenchfries,frenchwine,freshbread,freshtuna,fromageblanc,frozensmoothie,frozenvegetables,glutenfreebar,googletablet,gratedcheese,greenbeans,greengrapes,greentea,groundbeef,gums,half-life2,half-life:alyx,ham,handproteinbar,herb&pepper,honey,hotdogs,iMac,iPad,iphone8,ketchup,laptop,lightcream,lightmayo,lowfatyogurt,mashedpotato,mayonnaise,meatballs,megamanzero,megamanzero2,megamanzero3,megamanzero4,melons,metroidfusion,metroidprime,milk,minecraft,mineralwater,mint,mintgreentea,muffins,mushroomcreamsauce,napkins,nonfatmilk,oatmeal,oil,oliveoil,pancakes,parmesancheese,pasta,pepper,petfood,phonecarcharger,phonecharger,pickles,pokemonscarlet,pokemonshield,pokemonsword,pokemonviolet,portal,portal2,proteinbar,ratchet&clank,ratchet&clank2,ratchet&clank3,razor,redwine,rice,salad,salmon,salt,samsunggalaxy10,sandwich,seabass,shallot,shampoo,showergel,shrimp,soda,soup,spaghetti,sparklingwater,spinach,strawberries,strongcheese,tea,toiletpaper,tomatoes,tomatojuice,tomatosauce,toothbrush,toothpaste,trout,turkey,vacuumcleaner,vegetablesmix,waterspray,whitewine,wholeweatflour,wholewheatpasta,wholewheatrice,yams,yogurtcake,zucchini
0,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fals

In [31]:
creates_associations_rules(transaction_items=transactions_Young_electronics, min_support=0.1, min_confidence=0.5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
39,(soup),"(cookingoil, oil)",0.20,0.44,0.10,0.52,1.19,0.02,1.17,0.20
36,(napkins),"(cookingoil, oil)",0.23,0.44,0.12,0.51,1.17,0.02,1.16,0.19
35,"(napkins, oil)",(cookingoil),0.18,0.56,0.12,0.65,1.17,0.02,1.27,0.18
22,"(candybars, cake)",(cookingoil),0.16,0.56,0.10,0.65,1.16,0.01,1.26,0.17
38,"(oil, soup)",(cookingoil),0.16,0.56,0.10,0.65,1.16,0.01,1.25,0.16
29,(candybars),"(cookingoil, oil)",0.34,0.44,0.17,0.51,1.16,0.02,1.14,0.21
28,"(candybars, oil)",(cookingoil),0.27,0.56,0.17,0.64,1.15,0.02,1.23,0.18
12,(soup),(cookingoil),0.20,0.56,0.12,0.63,1.13,0.01,1.20,0.15
9,(napkins),(cookingoil),0.23,0.56,0.15,0.63,1.13,0.02,1.20,0.15
31,"(oil, gums)",(cookingoil),0.24,0.56,0.15,0.63,1.13,0.02,1.19,0.15


In [32]:
creates_associations_rules(transaction_items=transactions_Young_electronics, min_support=0.03, min_confidence=0.5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
320,"(candybars, soup)","(cookingoil, oil)",0.07,0.44,0.04,0.57,1.31,0.01,1.32,0.25
297,"(cake, soup)","(cookingoil, oil)",0.09,0.44,0.05,0.56,1.29,0.01,1.29,0.24
293,"(oliveoil, cake)","(cookingoil, oil)",0.06,0.44,0.04,0.56,1.28,0.01,1.28,0.24
317,"(candybars, napkins)","(cookingoil, oil)",0.09,0.44,0.05,0.56,1.27,0.01,1.27,0.23
195,"(oliveoil, gums)",(cookingoil),0.05,0.56,0.03,0.71,1.27,0.01,1.51,0.22
329,"(gums, soup)","(cookingoil, oil)",0.07,0.44,0.04,0.55,1.25,0.01,1.24,0.21
319,"(candybars, oil, soup)",(cookingoil),0.06,0.56,0.04,0.70,1.25,0.01,1.45,0.21
159,"(candybars, soup)",(cookingoil),0.07,0.56,0.05,0.70,1.25,0.01,1.45,0.21
298,"(cookingoil, oil, yogurtcake)",(cake),0.06,0.43,0.03,0.53,1.24,0.01,1.22,0.21
332,"(napkins, muffins)","(cookingoil, oil)",0.06,0.44,0.03,0.54,1.24,0.01,1.23,0.21


We had 5725 transactions made by the young electronics group, we considered 2 possible sets of rules, one with products that appear at least 10% of the time and 3% of the time, appearing at least 50% of the time together.
After looking at the results we see that:
+ These people bought mostly basic products like oil, napkins.
+ They buy candy in their basic baskets.

We suggest the following promotions and strategies to encourage purchases in this group:
+ Separate the oil and cooking oil from the napkins as much as possible, to make them to walk and see more products.
+ To encourage the purchase of other candies, put stands with candy bars and gummies near the checkout and the self-checkout.
+ To encourage them to buy cake: 20% discount on cake if they bought oil (or cooking oil).
+ To encourage higher purchases: 10% discount in electronic products on purchases higher than 20€.

Promotions are delivered in loyalty card and in ad campaigns.

## Basket Loyal Customers

In [33]:
basket_Loyal_Customers
# info of 11209 transactions

,list_of_goods,customer_id
invoice_id,,
10363735,"['extra dark chocolate', 'bluetooth headphones...",4514
7348902,"['milk', 'pepper', 'whole weat flour', 'cake',...",4514
3079886,"['megaman zero 4', 'oatmeal', 'whole wheat ric...",4514
1391035,"['black beer', 'cotton buds', 'napkins', 'nonf...",3231
6280179,"['milk', 'whole weat flour', 'gums', 'ratchet ...",3231
...,...,...
10027165,"['milk', 'yogurt cake', 'chocolate bread', 'co...",10816
8792457,"['whole wheat rice', 'toilet paper', 'gums', '...",31219
9691706,"['shampoo', 'cereals', 'final fantasy XX', 'wa...",13643


In [34]:
transactions_Loyal_Customers = dataframe_to_transaction_items(basket_Loyal_Customers)
transactions_Loyal_Customers

,airpods,almonds,antioxydantjuice,asparagus,avocado,babiesfood,bacon,barbecuesauce,beatsheadphones,beer,blackbeer,blacktea,blueberries,bluetoothheadphones,bodyspray,bramble,brownies,burgers,burgersauce,butter,cake,candybars,canned_tuna,carrots,catfish,cauliflower,cereals,champagne,chicken,chili,chocolate,chocolatebread,chutney,cider,cologne,cookies,cookingoil,corn,cottagecheese,cottonbuds,cream,deodorant,dessertwine,eggplant,eggs,energybar,energydrink,escalope,extradarkchocolate,finalfantasyXIX,finalfantasyXX,finalfantasyXXII,flaxseed,frenchfries,frenchwine,freshbread,freshtuna,fromageblanc,frozensmoothie,frozenvegetables,glutenfreebar,googletablet,gratedcheese,greenbeans,greengrapes,greentea,groundbeef,gums,half-life2,half-life:alyx,ham,handproteinbar,herb&pepper,honey,hotdogs,iMac,iPad,iphone8,ketchup,laptop,lightcream,lightmayo,lowfatyogurt,mashedpotato,mayonnaise,meatballs,megamanzero,megamanzero2,megamanzero3,megamanzero4,melons,metroidfusion,metroidprime,milk,minecraft,mineralwater,mint,mintgreentea,muffins,mushroomcreamsauce,napkins,nonfatmilk,oatmeal,oil,oliveoil,pancakes,parmesancheese,pasta,pepper,petfood,phonecarcharger,phonecharger,pickles,pokemonscarlet,pokemonshield,pokemonsword,pokemonviolet,portal,portal2,proteinbar,ratchet&clank,ratchet&clank2,ratchet&clank3,razor,redwine,rice,salad,salmon,salt,samsunggalaxy10,sandwich,seabass,shallot,shampoo,showergel,shrimp,soda,soup,spaghetti,sparklingwater,spinach,strawberries,strongcheese,tea,toiletpaper,tomatoes,tomatojuice,tomatosauce,toothbrush,toothpaste,trout,turkey,vacuumcleaner,vegetablesmix,waterspray,whitewine,wholeweatflour,wholewheatpasta,wholewheatrice,yams,yogurtcake,zucchini
0,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,True,False,False,False,False,False,False,False,True,True,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,T

In [35]:
creates_associations_rules(transaction_items=transactions_Loyal_Customers, min_support=0.1, min_confidence=0.5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
3,(soup),(cake),0.20,0.44,0.10,0.51,1.15,0.01,1.13,0.16
41,(muffins),"(cookingoil, oil)",0.25,0.47,0.13,0.54,1.15,0.02,1.15,0.17
37,"(oil, gums)",(cookingoil),0.25,0.57,0.16,0.65,1.14,0.02,1.23,0.17
38,(gums),"(cookingoil, oil)",0.30,0.47,0.16,0.54,1.14,0.02,1.14,0.18
25,"(candybars, oil)",(cake),0.29,0.44,0.14,0.50,1.14,0.02,1.12,0.17
31,"(oil, muffins)",(cake),0.21,0.44,0.11,0.50,1.14,0.01,1.12,0.15
47,(soup),"(cookingoil, oil)",0.20,0.47,0.11,0.53,1.13,0.01,1.13,0.14
43,"(napkins, oil)",(cookingoil),0.19,0.57,0.12,0.64,1.13,0.01,1.20,0.14
44,(napkins),"(cookingoil, oil)",0.23,0.47,0.12,0.53,1.12,0.01,1.12,0.14
40,"(oil, muffins)",(cookingoil),0.21,0.57,0.13,0.64,1.12,0.01,1.19,0.14


In [36]:
creates_associations_rules(transaction_items=transactions_Loyal_Customers, min_support=0.03, min_confidence=0.5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
325,"(candybars, oil, soup)",(cake),0.06,0.44,0.04,0.57,1.28,0.01,1.29,0.24
406,"(napkins, gums)","(cookingoil, oil)",0.07,0.47,0.04,0.60,1.28,0.01,1.33,0.23
397,"(freshbread, gums)","(cookingoil, oil)",0.06,0.47,0.04,0.60,1.27,0.01,1.32,0.23
400,"(gums, ketchup)","(cookingoil, oil)",0.05,0.47,0.03,0.59,1.26,0.01,1.30,0.22
415,"(muffins, soup)","(cookingoil, oil)",0.05,0.47,0.03,0.59,1.25,0.01,1.29,0.21
369,"(oil, gums, soup)",(cake),0.05,0.44,0.03,0.55,1.25,0.01,1.25,0.21
341,"(cake, ketchup)","(cookingoil, oil)",0.08,0.47,0.04,0.59,1.25,0.01,1.29,0.22
114,"(candybars, soup)",(cake),0.07,0.44,0.04,0.55,1.25,0.01,1.24,0.21
409,"(gums, soup)","(cookingoil, oil)",0.06,0.47,0.04,0.58,1.24,0.01,1.27,0.21
337,"(cake, gums)","(cookingoil, oil)",0.14,0.47,0.08,0.58,1.23,0.02,1.26,0.22


We had 11209 transactions made by the loyal customers group, we considered 2 possible sets of rules, one with products that appear at least 10% of the time and 3% of the time, appearing at least 50% of the time together.
After looking at the results we see that:
+ These people bought mostly basic products like oil, napkins.
+ They buy candy and soup in their basic baskets.

We suggest the following promotions and strategies to encourage purchases in this group:
+ Separate the oil and cooking oil from the napkins as much as possible, to make them to walk and see more products.
+ To incentivize the purchase of other candies, put stands with candy bars and gummies near the checkout and the self-checkout.
+ To incentivize them to buy soup: 20% discount on soup if they bought oil (or cooking oil).
+ To incentivize them to buy candy bars: 20% discount on candy bars if they bought cake.

Promotions are delivered using the loyalty card and tv commercials.


## Basket Promotion lovers


In [37]:
basket_Promotions
# info of 24536 transactions

,list_of_goods,customer_id
invoice_id,,
7448187,"['olive oil', 'yogurt cake', 'grated cheese', ...",43134
3167314,"['babies food', 'shower gel', 'rice', 'black b...",43134
10199712,"['babies food', 'barbecue sauce', 'olive oil',...",43134
12179179,"['babies food', 'milk', 'rice', 'yogurt cake',...",43134
8292388,"['napkins', 'cookies', 'razor', 'cooking oil',...",43134
...,...,...
6540039,"['bacon', 'napkins', 'phone car charger', 'cer...",10555
10074352,"['rice', 'beats headphones', 'ketchup', 'pet f...",31555
3649658,"['white wine', 'pet food', 'tomato sauce', 'br...",29709


In [38]:
transactions_Promotions = dataframe_to_transaction_items(basket_Promotions)
transactions_Promotions

,airpods,almonds,antioxydantjuice,asparagus,avocado,babiesfood,bacon,barbecuesauce,beatsheadphones,beer,blackbeer,blacktea,blueberries,bluetoothheadphones,bodyspray,bramble,brownies,burgers,burgersauce,butter,cake,candybars,canned_tuna,carrots,catfish,cauliflower,cereals,champagne,chicken,chili,chocolate,chocolatebread,chutney,cider,cologne,cookies,cookingoil,corn,cottagecheese,cottonbuds,cream,deodorant,dessertwine,eggplant,eggs,energybar,energydrink,escalope,extradarkchocolate,finalfantasyXIX,finalfantasyXX,finalfantasyXXII,flaxseed,frenchfries,frenchwine,freshbread,freshtuna,fromageblanc,frozensmoothie,frozenvegetables,glutenfreebar,googletablet,gratedcheese,greenbeans,greengrapes,greentea,groundbeef,gums,half-life2,half-life:alyx,ham,handproteinbar,herb&pepper,honey,hotdogs,iMac,iPad,iphone8,ketchup,laptop,lightcream,lightmayo,lowfatyogurt,mashedpotato,mayonnaise,meatballs,megamanzero,megamanzero2,megamanzero3,megamanzero4,melons,metroidfusion,metroidprime,milk,minecraft,mineralwater,mint,mintgreentea,muffins,mushroomcreamsauce,napkins,nonfatmilk,oatmeal,oil,oliveoil,pancakes,parmesancheese,pasta,pepper,petfood,phonecarcharger,phonecharger,pickles,pokemonscarlet,pokemonshield,pokemonsword,pokemonviolet,portal,portal2,proteinbar,ratchet&clank,ratchet&clank2,ratchet&clank3,razor,redwine,rice,salad,salmon,salt,samsunggalaxy10,sandwich,seabass,shallot,shampoo,showergel,shrimp,soda,soup,spaghetti,sparklingwater,spinach,strawberries,strongcheese,tea,toiletpaper,tomatoes,tomatojuice,tomatosauce,toothbrush,toothpaste,trout,turkey,vacuumcleaner,vegetablesmix,waterspray,whitewine,wholeweatflour,wholewheatpasta,wholewheatrice,yams,yogurtcake,zucchini
0,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False
1,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,True,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
2,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,Fal

In [39]:
creates_associations_rules(transaction_items=transactions_Promotions, min_support=0.12, min_confidence=0.5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
4,(cider),(whitewine),0.19,0.26,0.15,0.76,2.93,0.10,3.07,0.82
5,(whitewine),(cider),0.26,0.19,0.15,0.56,2.93,0.10,1.86,0.89
15,"(candybars, oil)",(cookingoil),0.21,0.42,0.13,0.63,1.51,0.04,1.58,0.43
13,"(oil, cake)",(cookingoil),0.26,0.42,0.16,0.63,1.51,0.05,1.57,0.45
14,"(cookingoil, candybars)",(oil),0.16,0.57,0.13,0.85,1.50,0.04,2.87,0.39
12,"(cookingoil, cake)",(oil),0.19,0.57,0.16,0.85,1.49,0.05,2.81,0.41
6,(gums),(cookingoil),0.23,0.42,0.13,0.58,1.39,0.04,1.39,0.36
2,(candybars),(cookingoil),0.27,0.42,0.16,0.58,1.39,0.04,1.39,0.38
7,(cookingoil),(oil),0.42,0.57,0.33,0.79,1.38,0.09,2.01,0.48
8,(oil),(cookingoil),0.57,0.42,0.33,0.58,1.38,0.09,1.38,0.64


In [40]:
creates_associations_rules(transaction_items=transactions_Promotions, min_support=0.06, min_confidence=0.5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
46,"(cider, dessertwine)",(whitewine),0.08,0.26,0.07,0.85,3.29,0.05,4.99,0.76
47,"(whitewine, dessertwine)",(cider),0.11,0.19,0.07,0.63,3.26,0.05,2.16,0.78
10,(cider),(whitewine),0.19,0.26,0.15,0.76,2.93,0.10,3.07,0.82
11,(whitewine),(cider),0.26,0.19,0.15,0.56,2.93,0.10,1.86,0.89
22,(dessertwine),(whitewine),0.15,0.26,0.11,0.75,2.89,0.07,2.94,0.77
2,(beer),(whitewine),0.12,0.26,0.09,0.74,2.86,0.06,2.85,0.74
9,(dessertwine),(cider),0.15,0.19,0.08,0.55,2.86,0.05,1.80,0.77
1,(beer),(cider),0.12,0.19,0.07,0.55,2.86,0.04,1.80,0.74
8,(champagne),(whitewine),0.11,0.26,0.08,0.73,2.83,0.05,2.77,0.72
3,(bramble),(whitewine),0.10,0.26,0.07,0.70,2.73,0.04,2.51,0.70


We had 24536 transactions made by the lpromotions customers group, we considered 2 possible sets of rules, one with products that appear at least 12% of the time and 6% of the time, appearing at least 50% of the time together.
After looking at the results we see that:
+ There are people that buy only basic products as napkins and oil, and people that buy alcohol (cider, beer and wine).
+ These people are highly sensitive to promotions.

We suggest the following promotions and strategies to encourage purchases in this group:
+ To encourage higher purchases: 20% of discounts on purchases higher than 100€.
+ To encourage them to buy:  Specific 10% discount on a certain product category (pets, vegetables, meats, fish, electronics, video games, alcohol and non-alcohol and hygiene).
+ To encourage them to buy wine products together: 20% discount on dessert wine and white wine if bought them together.
+ To incentivize the purchase of other products: 15% discount on beer on the purchase of cider (apply the same discount to white wine and dessert wine).
+ To incentivize the purchase of other candies, put stands with candy bars and gummies near the checkout and the self-checkout.

Promotions are delivered using the loyalty card and tv commercials.

## Basket Parents

In [41]:
basket_Parents
# info of 905 transactions

,list_of_goods,customer_id
invoice_id,,
11084556,"['toilet paper', 'cooking oil', 'french fries'...",25564
9580728,"['babies food', 'chocolate bread', 'fresh brea...",25564
1713818,"['babies food', 'cologne', 'olive oil', 'butte...",25564
2523382,"['babies food', 'beer', 'gums', 'cooking oil',...",25564
8881202,"['babies food', 'megaman zero 4', 'frozen smoo...",25564
...,...,...
9735264,"['babies food', 'milk', 'iPad', 'gums', 'oil']",8675
10501212,"['rice', 'olive oil', 'butter', 'cotton buds',...",28346
7405407,"['whole wheat rice', 'ground beef', 'cottage c...",16788


In [42]:
transactions_Parents = dataframe_to_transaction_items(basket_Parents)
transactions_Parents

,airpods,almonds,antioxydantjuice,asparagus,avocado,babiesfood,bacon,barbecuesauce,beatsheadphones,beer,blackbeer,blacktea,blueberries,bluetoothheadphones,bodyspray,bramble,brownies,burgers,burgersauce,butter,cake,candybars,canned_tuna,carrots,catfish,cauliflower,cereals,champagne,chicken,chili,chocolate,chocolatebread,chutney,cider,cologne,cookies,cookingoil,corn,cottagecheese,cottonbuds,cream,deodorant,dessertwine,eggplant,eggs,energybar,energydrink,escalope,extradarkchocolate,finalfantasyXIX,finalfantasyXX,finalfantasyXXII,flaxseed,frenchfries,frenchwine,freshbread,freshtuna,fromageblanc,frozensmoothie,frozenvegetables,glutenfreebar,googletablet,gratedcheese,greenbeans,greengrapes,greentea,groundbeef,gums,half-life2,half-life:alyx,ham,handproteinbar,herb&pepper,honey,hotdogs,iMac,iPad,iphone8,ketchup,laptop,lightcream,lightmayo,lowfatyogurt,mashedpotato,mayonnaise,meatballs,megamanzero,megamanzero2,megamanzero3,megamanzero4,melons,metroidfusion,metroidprime,milk,minecraft,mineralwater,mint,mintgreentea,muffins,mushroomcreamsauce,napkins,nonfatmilk,oatmeal,oil,oliveoil,pancakes,parmesancheese,pasta,pepper,petfood,phonecarcharger,phonecharger,pickles,pokemonscarlet,pokemonshield,pokemonsword,pokemonviolet,portal,portal2,proteinbar,ratchet&clank,ratchet&clank2,ratchet&clank3,razor,redwine,rice,salad,salmon,salt,samsunggalaxy10,sandwich,seabass,shallot,shampoo,showergel,shrimp,soda,soup,spaghetti,sparklingwater,spinach,strawberries,strongcheese,tea,toiletpaper,tomatoes,tomatojuice,tomatosauce,toothbrush,toothpaste,trout,turkey,vacuumcleaner,vegetablesmix,waterspray,whitewine,wholeweatflour,wholewheatpasta,wholewheatrice,yams,yogurtcake,zucchini
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,True,True,False,False,False,False,False,False,True,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fa

In [43]:
creates_associations_rules(transaction_items=transactions_Parents, min_support=0.15, min_confidence=0.5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
20,(gums),"(cookingoil, babiesfood)",0.30,0.47,0.16,0.53,1.14,0.02,1.14,0.18
19,"(babiesfood, gums)",(cookingoil),0.25,0.57,0.16,0.64,1.13,0.02,1.21,0.15
14,(cake),"(cookingoil, babiesfood)",0.43,0.47,0.23,0.52,1.12,0.02,1.12,0.19
17,(candybars),"(cookingoil, babiesfood)",0.36,0.47,0.19,0.52,1.11,0.02,1.11,0.15
13,"(babiesfood, cake)",(cookingoil),0.36,0.57,0.23,0.63,1.11,0.02,1.16,0.15
11,(muffins),(cookingoil),0.26,0.57,0.16,0.63,1.10,0.01,1.15,0.12
16,"(candybars, babiesfood)",(cookingoil),0.30,0.57,0.19,0.63,1.10,0.02,1.15,0.13
10,(gums),(cookingoil),0.30,0.57,0.19,0.63,1.10,0.02,1.15,0.12
15,"(cookingoil, candybars)",(babiesfood),0.22,0.78,0.19,0.85,1.09,0.02,1.50,0.11
18,"(cookingoil, gums)",(babiesfood),0.19,0.78,0.16,0.85,1.09,0.01,1.47,0.10


In [44]:
creates_associations_rules(transaction_items=transactions_Parents, min_support=0.08, min_confidence=0.5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
73,"(cake, gums)","(cookingoil, babiesfood)",0.14,0.47,0.08,0.57,1.22,0.01,1.23,0.21
69,"(candybars, cake)","(cookingoil, babiesfood)",0.17,0.47,0.09,0.55,1.17,0.01,1.18,0.18
65,"(cake, muffins)",(cookingoil),0.12,0.57,0.08,0.66,1.16,0.01,1.27,0.16
72,"(babiesfood, cake, gums)",(cookingoil),0.12,0.57,0.08,0.66,1.16,0.01,1.27,0.16
70,"(cookingoil, babiesfood, gums)",(cake),0.16,0.43,0.08,0.50,1.16,0.01,1.14,0.16
67,"(cookingoil, candybars, babiesfood)",(cake),0.19,0.43,0.09,0.50,1.16,0.01,1.14,0.17
59,(napkins),"(cookingoil, babiesfood)",0.23,0.47,0.12,0.53,1.14,0.02,1.14,0.16
50,(gums),"(cookingoil, babiesfood)",0.30,0.47,0.16,0.53,1.14,0.02,1.14,0.18
56,(muffins),"(cookingoil, babiesfood)",0.26,0.47,0.14,0.53,1.14,0.02,1.14,0.17
64,"(cake, gums)",(cookingoil),0.14,0.57,0.09,0.65,1.14,0.01,1.22,0.14


We had 905 transactions made by the loyal customers group, we considered 2 possible sets of rules, one with products that appear at least 15% of the time and 8% of the time, appearing at least 50% of the time together.
After looking at the results we see that:
+ These people buy basic products like oil and candies and baby food.
+ These people have many children.

We suggest the following promotions and strategies to encourage purchases in this group:
+ To encourage the purchase of toys: buy 1 get 1 at 50%.
+ Separate the oil and cooking oil from the baby food as much as possible, to incentivize them to walk and see more products.
+ To incentivize the purchase of other candies, put stands with candy bars and gummies near the checkout and the self-checkout.

Promotions are delivered using the loyalty card and tv commercials.

# Conclusions
If we were to choose which customers should be targeted first and which strategies to be considered more seriously, the order will be the following:
1) **Fishermen:** because they are easy to contact and they spent a lot in our store.
2) **Gamers:** because they are the second most profitable clients.
3) **Pet lovers:** because they are easy to satisfy and they are the next in the list of spending more.
4) **Promotions (and young parties):** because they have rules from the young parties group and are the most sensitive to promotions.
5) **Parents:** because the strategies don't imply ads (only 1 but it could be omitted).
6) **Vegetarians:** All promotions to compete for that market group.
7) **Young lots of electronics:** only the strategic decisions and the promotion on electronics.
8) **Loyal customers:** Nothing, they won’t leave because changing habits is hard.

